# Lectura, Limpieza y Exportación de Datos con Pandas

## Objetivos

1. **Cargar datasets desde diferentes formatos** (CSV, JSON, Excel, SQL, etc.) utilizando parámetros adecuados (`sep`, `na_values`, `parse_dates`, etc.).  
2. **Detectar y tratar valores faltantes y sucios** (`NaN`, `NULL`, `?`, vacíos), transformándolos en un formato uniforme.  
3. **Ajustar tipos de datos** (fechas, números, cadenas, IDs, etc.) para que el dataset sea más consistente.  
4. **Aplicar análisis preliminar** (`info()`, `head()`, `describe()`, `nunique()`) para conocer la estructura y calidad de los datos.  
5. **Exportar el DataFrame limpio en múltiples formatos** (CSV, Excel, JSON, SQL, y opcionalmente binarios como Pickle/Parquet/HDF5).  
6. **Organizar las exportaciones en carpetas dedicadas** para mantener un flujo de trabajo ordenado. 


##  Librerías


In [94]:
import json, time, math, sys, os
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize
from urllib.parse import urlparse
import sqlite3


## 1) Lectura de dataset desde una fuente externa o local



In [95]:

def cargar_dataset(src, *, cols=None):
    """
    Carga un dataset desde:
    - API pública (http/https con JSON)
    - Archivo JSON local
    - Archivo CSV local
    """
    parsed = urlparse(src)

    # Caso 1: API pública
    if parsed.scheme in ("http", "https"):
        r = requests.get(src, timeout=10)
        r.raise_for_status()
        data = r.json()
        df = json_normalize(data, sep=".")

    # Caso 2: JSON local
    elif src.lower().endswith(".json") and os.path.isfile(src):
        with open(src, "r", encoding="utf-8") as f:
            data = json.load(f)
        df = json_normalize(data, sep=".")

    # Caso 3: CSV local
    elif src.lower().endswith(".csv") and os.path.isfile(src):
        df = pd.read_csv(src, sep=";", encoding="utf-8")

    # Ningún formato válido
    else:
        raise ValueError(f"❌ No existe un formato válido para la fuente: {src}")

    # Selección opcional de columnas
    if cols:
        keep = [c for c in cols if c in df.columns]
        df = df[keep]

    return df


# ==================================================
# 🔹 Elegí la fuente
# ==================================================

# API pública
#df = cargar_dataset("https://fakestoreapi.com/products")

# JSON local
#df = cargar_dataset("json/fake_products.json")

# CSV local
df = cargar_dataset("datasets/car_data.csv")

df.head() #Te muestra los nombres de columnas y los primeros registros, sin necesidad de imprimir todo el dataset.

,Car_id,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Dealer_No,Body Style,Phone,Dealer_Region
0,C_CND_000001,1/2/2022,Geraldine,Male,13500.0,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000.0,06457-3834,SUV,8264678.0,Middletown
1,C_CND_000002,1/2/2022,Gia,Male,1480000.0,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000.0,60504-7114,SUV,6848189.0,Aurora
2,C_CND_000003,1/2/2022,Gianna,Male,1035000.0,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500.0,38701-8047,Passenger,7298798.0,Greenville
3,C_CND_000004,1/2/2022,Giselle,Male,13500.0,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000.0,99301-3882,SUV,6257557.0,Pasco
4,C_CND_000005,1/2/2022,Grace,Male,1465000.0,Chrysler Plymouth,Acura,TL,DoubleÂ Overhead Camshaft,Auto,Red,24500.0,53546-9427,Hatchback,7081483.0,Janesville


In [96]:

df.info()          # tipos de datos detectados


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23879 entries, 0 to 23878
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_id         111 non-null    object 
 1   Date           111 non-null    object 
 2   Customer Name  111 non-null    object 
 3   Gender         111 non-null    object 
 4   Annual Income  111 non-null    float64
 5   Dealer_Name    111 non-null    object 
 6   Company        111 non-null    object 
 7   Model          111 non-null    object 
 8   Engine         111 non-null    object 
 9   Transmission   111 non-null    object 
 10  Color          111 non-null    object 
 11  Price ($)      111 non-null    float64
 12  Dealer_No      111 non-null    object 
 13  Body Style     111 non-null    object 
 14  Phone          111 non-null    float64
 15  Dealer_Region  111 non-null    object 
dtypes: float64(3), object(13)
memory usage: 2.9+ MB


# Limpieza de Datos

## Descripción

La **limpieza de datos** es una etapa clave en cualquier análisis. Permite transformar la información en un formato uniforme y listo para explorar.  
Las tareas más comunes incluyen:

-  **Detectar y manejar valores faltantes** (`NaN`, `NULL`, `?`, `N/A`, etc.).  
-  **Convertir cadenas de texto a tipos adecuados** (números, fechas, booleanos).  
-  **Eliminar símbolos innecesarios** (ejemplo: `$`, `,`).  
-  **Uniformar formatos** (especialmente en fechas).  



In [97]:
import pandas as pd

# Cargar el dataset con la configuración correcta
df = pd.read_csv(
    "datasets/car_data.csv",
    sep=";",                           # ¡CORREGIDO! El archivo usa punto y coma, no coma
    na_values=["NA", "NULL", "?", ""], # normalizamos nulos
    parse_dates=["Date"],              # columna de fechas
    thousands=",",                     # separador de miles en Price ($)
    decimal=".",                       # separador decimal (estándar en inglés)
    dtype={
        "Car_id": "string",            # ¡CORREGIDO! Car_id es alfanumérico (ej: "C_CND_000001")
        "Dealer_No": "string",         # códigos de dealer como string
        "Phone": "string"              # teléfono mejor como string
    }
)

# Limpiar espacios en los nombres de columnas (después de cargar)
df.columns = df.columns.str.strip()

print("✅ Dataset cargado correctamente")
print(f"📏 Dimensiones: {df.shape}")
print(f"📋 Columnas: {list(df.columns)}")
df.head()

✅ Dataset cargado correctamente
📏 Dimensiones: (23879, 16)
📋 Columnas: ['Car_id', 'Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Dealer_No', 'Body Style', 'Phone', 'Dealer_Region']


,Car_id,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Dealer_No,Body Style,Phone,Dealer_Region
0,C_CND_000001,2022-01-02,Geraldine,Male,13500.0,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000.0,06457-3834,SUV,8264678,Middletown
1,C_CND_000002,2022-01-02,Gia,Male,1480000.0,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000.0,60504-7114,SUV,6848189,Aurora
2,C_CND_000003,2022-01-02,Gianna,Male,1035000.0,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500.0,38701-8047,Passenger,7298798,Greenville
3,C_CND_000004,2022-01-02,Giselle,Male,13500.0,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000.0,99301-3882,SUV,6257557,Pasco
4,C_CND_000005,2022-01-02,Grace,Male,1465000.0,Chrysler Plymouth,Acura,TL,DoubleÂ Overhead Camshaft,Auto,Red,24500.0,53546-9427,Hatchback,7081483,Janesville


In [98]:
#  Ejemplo: Limpiar la columna de precios

print("🔍 Antes de limpiar:")
print("Ejemplos de precios:", df["Price ($)"].head().tolist())
print("Tipo de dato:", df["Price ($)"].dtype)

# Limpiar símbolos $ y comas, convertir a número
df["price_clean"] = df["Price ($)"].str.replace(r"[\$,]", "", regex=True)
df["price_clean"] = pd.to_numeric(df["price_clean"], errors="coerce")

print("\n✅ Después de limpiar:")
print("Ejemplos de precios:", df["price_clean"].head().tolist())
print("Tipo de dato:", df["price_clean"].dtype)
print("Precio promedio:", f"${df['price_clean'].mean():,.0f}")

# Comparación lado a lado
comparison = df[["Price ($)", "price_clean"]].head()
print("\n📊 Comparación:")
comparison

🔍 Antes de limpiar:
Ejemplos de precios: [26000.0, 19000.0, 31500.0, 14000.0, 24500.0]
Tipo de dato: float64


AttributeError: Can only use .str accessor with string values!

## 🔍 Exploración rápida de los datos

Ahora que tenemos el dataset cargado, vamos a explorarlo de forma básica para entender qué contiene:

### Preguntas básicas que siempre debemos hacer:
1. **¿Cuántas filas y columnas tiene?** → `df.shape`
2. **¿Qué tipos de datos hay?** → `df.info()`
3. **¿Hay valores faltantes?** → `df.isnull().sum()`
4. **¿Cómo se ven las primeras filas?** → `df.head()`
5. **¿Qué valores únicos hay en columnas categóricas?** → `df['columna'].value_counts()`

In [ ]:
# 🔍 Exploración básica del dataset

print("1️⃣ Información general del dataset:")
print(f"   📏 Filas: {df.shape[0]:,}")
print(f"   📋 Columnas: {df.shape[1]}")
print(f"   💾 Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\n2️⃣ Tipos de datos:")
print(df.dtypes)

print("\n3️⃣ Valores faltantes:")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(1)
missing_report = pd.DataFrame({
    'Faltantes': missing,
    'Porcentaje': missing_pct
}).sort_values('Porcentaje', ascending=False)
print(missing_report[missing_report['Faltantes'] > 0])

print("\n4️⃣ Primeras 3 filas:")
df.head(3)

##  Limpieza básica de datos

Los datos "crudos" siempre necesitan limpieza. Veamos los problemas más comunes y cómo resolverlos:

### Problemas típicos:
- 💲 **Precios con símbolos**: `"$45,000"` → debe ser `45000` (número)
- 📅 **Fechas inconsistentes**: diferentes formatos
- 🔤 **Espacios extra** en nombres o categorías
- ❓ **Valores faltantes** codificados como texto (`"NA"`, `"?"`, `""`)
- 🔢 **Números como texto** por culpa de símbolos o separadores

In [ ]:

# 📤 Exportación del DataFrame limpio en distintos formatos (carpeta export/)



# Crear carpeta "export" si no existe
os.makedirs("export", exist_ok=True)

# 1) CSV
df.to_csv("export/datos_limpios.csv", index=False, encoding="utf-8")

# 2) Excel
df.to_excel("export/datos_limpios.xlsx", index=False)

# 3) JSON
df.to_json("export/datos_limpios.json", orient="records", indent=2, force_ascii=False)

# 4) SQL (ejemplo con SQLite)
con = sqlite3.connect("export/datos.db")
df.to_sql("tabla_limpia", con, if_exists="replace", index=False)
con.close()